# Chapter 12 - K Nearest Neighbors 

### The Model
#### import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List
from collections import Counter
from typing import NamedTuple
import requests

In [2]:
def raw_majority_vote(labels: List[str]) -> str:
    votes = Counter(labels)
    winner, _ = votes.most_common(1)[0]
    return winner

In [3]:
assert raw_majority_vote(['a', 'b', 'c', 'b']) == 'b'

In [4]:
def majority_vote(labels: List[str]) -> str:
    vote_counts = Counter(labels)
    winner, winner_count = vote_counts.most_common(1)[0]
    num_winners = len([count
                       for count in vote_counts.values()
                       if count == winner_count])
    if num_winners == 1:
        return winner
    else:
        return majority_vote(['a', 'b', 'c', 'b', 'a']) == 'b'

In [5]:
Vector = List[float]

def dot(v: Vector, w: Vector) -> float:
    """Computes v_1 * w_1 + ... + v_n * w_n"""
    assert len(v) == len(w), "vectors must be same length"

    return sum(v_i * w_i for v_i, w_i in zip(v, w))

assert dot([1, 2, 3], [4, 5, 6]) == 32  # 1 * 4 + 2 * 5 + 3 * 6

def sum_of_squares(v: Vector) -> float:
    """Returns v_1 * v_1 + ... + v_n * v_n"""
    return dot(v, v)


def squared_distance(v: Vector, w: Vector) -> float:
    """Computes (v_1 - w_1) ** 2 + ... + (v_n - w_n) ** 2"""
    return sum_of_squares(subtract(v, w))

def distance(v: Vector, w: Vector) -> float:
    """Computes the distance between v and w"""
    return math.sqrt(squared_distance(v, w))

In [6]:
# Setting Class LabeledPoint

class LabeledPoint(NamedTuple):
    point: Vector
    label: str
        
def knn_classify(k: int,
                 labeled_points: List[LabeledPoint],
                 new_point: Vector) -> str:
    
    by_distance = sorted(labeled_points,
                         key=lambda lp: distance(lp.point, new_point))
    
    k_nearest_labels = [lp.label for lp in by_distance[:k]]
    
    return majority_vote(k_nearest_labels)

### Example: The iris Dataset

In [30]:
data = requests.get(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
)

In [31]:
with open('iris.data', 'w') as f:
    f.write(data.text)

In [32]:
from typing import Dict
import csv
from collections import defaultdict

In [34]:
# def parse_iris_row(row: List[str]) -> LabeledPoint:
#     measurements = [float(value) for value in row[:-1]]
#     label = row[-1].split("-")[-1]
    
#     return LabeledPoint(measurements, label)

# with open('iris.data') as f:
#     reader = csv.reader(f)
#     iris_data = [parse_iris_row(row) for row in reader]
    
# points_by_species: Dict[str, List[Vector]] = defaultdict(list)
# for iris in iris_data:
#     points_by_species[iris.label].append(iris.point)

In [19]:
# metrics = [' sepal length', 'sepal width', 'petal length', 'petal width']
# pairs = [(i, j) for i in range(4) for j in range(4) if i < j]
# marks = ['+', '.', 'x']

# fig, ax = plt.subplot(2, 3)

# for row in range(2):
#     for col in range(3):
#         i, j = pairs[3 * row + col]
#         ax[row][col].set_title(f"{metrics[i]} vs {metrics[j]}", fontsize=8)
#         ax[row][col].set_xticks([])
#         ax[row][col].set_yticks([])
        
#         for mark, (species, points) in zip(marks, points_by_species.items()):
#             xs = [point[i] for point in points]
#             ys = [point[j] for point in points]
#             ax[row][col].scatter(xs, ys, marker=mark, label=species)
            
# ax[-1][-1].legend(loc='lower right', prop={'size': 6})

In [20]:
import random
from typing import TypeVar, List, Tuple

X = TypeVar('X')  # generic type to represent a data point

def split_data(data: List[X], prob: float) -> Tuple[List[X], List[X]]:
    """Split data into fractions [prob, 1 - prob]"""
    data = data[:]                    # Make a shallow copy
    random.shuffle(data)              # because shuffle modifies the list.
    cut = int(len(data) * prob)       # Use prob to find a cutoff
    return data[:cut], data[cut:]     # and split the shuffled list there.

data = [n for n in range(1000)]
train, test = split_data(data, 0.75)

# The proportions should be correct
assert len(train) == 750
assert len(test) == 250

# And the original data should be preserved (in some order)
assert sorted(train + test) == data

Y = TypeVar('Y')  # generic type to represent output variables

In [21]:
# random.seed(12)
# iris_train, iris_test = split_data(iris_data, 0.70)
# assert len(iris_train) == 0.7 * 150
# assert len(iris_test) == 0.3 * 150

In [22]:
from typing import Tuple

In [24]:
# confusion_matrix: Dict[Tuple[str, str], int] = defaultdict(int)
# num_correct = 0

# for iris in iris_test:
#     predicted = knn_classify(5, lris_train, iris.point)
#     actual = iris.label
    
#     if predicted == actual:
#         num_correct += 1
        
#     confusion_matrix[(predicted, actual)] += 1 
    
# pct_correct = num_correct / len(iris_test)
# print(pct_correct, cofusion_matrix)

### Curse of Dimensionality

In [25]:
def random_point(dim: int) -> Vector:
    return [random.random() for _ in range(dim)]

def random_distance(dim: int, num_pairs: int) -> List[float]:
    return [distance(random_point(dim), random_point(dim))
            for _ in range(num_pairs)]

In [27]:
# import tqdm
# dimensions = range(1, 10)

# avg_distances = []
# min_distances = []

# random.seed(0)
# for dim in tqdm(dimensions, desc="Curse of Dimensionality"):
#     distances = random_distances(dim, 10000)
#     avg_distances.append(sum(distances) / 10000)
#     min_distances.append(min(distances))

In [28]:
# min_avg_ratio = [min_dist / avg_dist
#                  for_min_dist, avg_dist in zip(min_distances, avg_distances)]